In [ ]:
/*

At forstå hvad er det der går tit i stykker (per modelnummer, equipment art og producent)
Hvor i landet går ting konstant i stykker
Hvor mange arbejdstimer forbruger vi hver måned
Hvilke kompetencer har vi tit brug for når der er noget der går i stykker
Fra hvilke afdelinger vi har brugt medarbejdere og timer så vi kan ”betale” dem.

Ideelt skal vi have en hjemmeside eller en App hvor man kan tilgå databasen for at indtaste noget nyt eller redigere. SAP sælger en sådan GUI som gør det nemt for brugeren, men man kan lave sin egen
hjemmeside eller bruge en Platforme som PowerApps. 


Remember:
-diagram
-find alle driftsrapporter en medarbejder har arbejdet på (2 Queries on top of eachother (webinar uge4 41:00), det hjælper softwareudvikleren med at spare tid - reduntancy)
- API(webinar uge4 sto telos )
-Stored procedure: svært at inject sql
-indexing (Chapters 3,4)



*/

# 1\. Forespørgsler

## 1.1 TOP 10 modelnumre der gik mest i stykker i 2017

In [11]:
-- TOP 10 modelnumre der gik mest i stykker i 2017 (sorteret for antallet af gange hver modelnummer gik i stykker)


-- Drop the stored procedure called 'stp_getAntaliStykkerPerModelnummer' in schema 'dbo'
USE [Eksamensopgave_jernbaneforvalter]

IF EXISTS (
SELECT *
    FROM INFORMATION_SCHEMA.ROUTINES
WHERE SPECIFIC_SCHEMA = N'dbo'
    AND SPECIFIC_NAME = N'stp_getAntaliStykkerPerModelnummer'
)
DROP PROCEDURE dbo.stp_getAntaliStykkerPerModelnummer
GO

-- Create a new stored procedure
CREATE PROCEDURE stp_getAntaliStykkerPerModelnummer @AntalRaekker INT, @aar INT
AS
    ;WITH 
    tempCountDriftsrapporter AS (
        SELECT Modelnummer, COUNT(*) AS AntalGangeiStykker
        FROM [Eksamensopgave_jernbaneforvalter].[dbo].[alledriftsrapportertabel]
        WHERE YEAR(Fejlmeldingsdato_og_tid) = @aar
        GROUP BY Modelnummer
    )
    SELECT TOP(@AntalRaekker) WITH TIES
        tmpa.Modelnummer, 
        e.Equipmentart,
        p.Producent,
        tmpa.AntalGangeiStykker
    FROM tempCountDriftsrapporter AS tmpa
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[allemodelnumretabel] AS m ON tmpa.Modelnummer = m.Modelnummer
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[equipmentarttabel] AS e ON m.EquipmentartID = e.EquipmentartID
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[producenttabel] AS p ON m.ProducentID = p.ProducentID
    ORDER BY tmpa.AntalGangeiStykker DESC
GO

EXEC stp_getAntaliStykkerPerModelnummer @AntalRaekker = 10, @aar = 2017;

Commands completed successfully.

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.010

Modelnummer,Equipmentart,Producent,AntalGangeiStykker
B22112H,Stationssikringsanlæg,Einstein Rosen Bridge Infrastracture,1534
B22096H,Signal,Neptune Train Corp,908
B22085G,Overkørselsanlæg,Einstein Rosen Bridge Infrastracture,891
B22111H,Stationssikringsanlæg,Neptune Train Corp,826
B22084G,Overkørselsanlæg,Neptune Train Corp,795
B22120H,Togdetektering,Neptune Train Corp,788
B22121H,Togdetektering,Einstein Rosen Bridge Infrastracture,524
B22083G,Overkørselsanlæg,Sagittarius A star Industries,397
B22104H,Sporskiftedrev,Sagittarius A star Industries,370
B22110H,Stationssikringsanlæg,Sagittarius A star Industries,353


## 1.2 TOP 10 udstyrstyper der gik mest i stykker i 2017

In [12]:
-- TOP 10 Equipmentart (udstyrstyper) der gik mest i stykker i 2017 (sorteret for antallet af gange hver Equipmentart gik i stykker)

-- Drop the stored procedure called 'stp_getAntaliStykkerPerEquipmentart' in schema 'dbo'
USE [Eksamensopgave_jernbaneforvalter]

IF EXISTS (
SELECT *
    FROM INFORMATION_SCHEMA.ROUTINES
WHERE SPECIFIC_SCHEMA = N'dbo'
    AND SPECIFIC_NAME = N'stp_getAntaliStykkerPerEquipmentart'
)
DROP PROCEDURE dbo.stp_getAntaliStykkerPerEquipmentart
GO

-- Create a new stored procedure
CREATE PROCEDURE stp_getAntaliStykkerPerEquipmentart @AntalRaekker INT, @aar INT
AS 
    SELECT TOP(@AntalRaekker) WITH TIES
        e.Equipmentart,
        COUNT(*) AS AntalGangeiStykker
    FROM [Eksamensopgave_jernbaneforvalter].[dbo].[alledriftsrapportertabel] AS a
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[allemodelnumretabel] AS m ON a.Modelnummer = m.Modelnummer
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[equipmentarttabel] AS e ON m.EquipmentartID = e.EquipmentartID
    WHERE YEAR(a.Fejlmeldingsdato_og_tid) = @aar
    GROUP BY e.Equipmentart
    ORDER BY AntalGangeiStykker DESC
GO

EXEC stp_getAntaliStykkerPerEquipmentart @AntalRaekker = 10, @aar = 2017;


Commands completed successfully.

Commands completed successfully.

(10 rows affected)

Total execution time: 00:00:00.024

Equipmentart,AntalGangeiStykker
Stationssikringsanlæg,2713
Overkørselsanlæg,2083
Signal,1594
Togdetektering,1488
Balise,810
Sporskiftedrev,801
Server,357
Betjeningsapparat,220
Bom/Bomdrev,152
Fjernstyringsunderst,88


## 1.3 Søg en medarbejder for at se deres jobfunktion, afdeling, leder og deres månedlige antal arbejdstimer i 2017

### 1.3.1 Find først den rigtige medarbejder (ved at slå op deres navn eller nummer)

In [72]:
-- Find først medarbejdernumret hvis man ikke kender det ved at søge medarbejderens navn eller nummer

-- Drop the stored procedure called 'stp_soegmedarbejder' in schema 'dbo'
USE [Eksamensopgave_jernbaneforvalter]

IF EXISTS (
SELECT *
    FROM INFORMATION_SCHEMA.ROUTINES
WHERE SPECIFIC_SCHEMA = N'dbo'
    AND SPECIFIC_NAME = N'stp_soegmedarbejder'
)
DROP PROCEDURE dbo.stp_soegmedarbejder
GO

-- Create a new stored procedure
CREATE PROCEDURE stp_soegmedarbejder @Navn NVARCHAR(50), @Nummer INT
AS 
    -- Hvis man søger medarbejderens navn
    IF (@Nummer = 0) 
    BEGIN
        ;WITH
        AfdWithMng AS(
            SELECT 
                afd.AfdelingID,
                afd.Afdeling,
                ISNULL(mrb.Title + ' ', '') + ISNULL(mrb.Navn + ' ', '') + ISNULL(mrb.Middelnavn + ' ', '') + ISNULL(mrb.Efternavn + ' ', '') AS [LederensNavn]
            FROM [Eksamensopgave_jernbaneforvalter].[dbo].[afdelingtabel] AS afd
            LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[allemedarbejderetabel] AS mrb ON afd.Leder = mrb.Medarbejdernummer
        ),
        tempMedrbtabel AS(
            SELECT a.Medarbejdernummer, 
                ISNULL(a.Title + ' ', '') + ISNULL(a.Navn + ' ', '') + ISNULL(a.Middelnavn + ' ', '') + ISNULL(a.Efternavn + ' ', '') AS [Fuldnavn],
                j.Jobfunktion,
                afmgr.Afdeling,
                afmgr.LederensNavn
            FROM [Eksamensopgave_jernbaneforvalter].[dbo].[allemedarbejderetabel] AS a
            LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[jobfunktiontabel] AS j ON a.JobfunktionID = j.JobfunktionID
            LEFT OUTER JOIN AfdWithMng AS afmgr ON a.AfdelingID = afmgr.AfdelingID
        )
        SELECT tmpM.Medarbejdernummer, 
            tmpM.Fuldnavn,
            tmpM.Jobfunktion,
            tmpM.Afdeling,
            tmpM.LederensNavn
        FROM tempMedrbtabel AS tmpM
        WHERE tmpM.Fuldnavn LIKE '%' + @Navn + '%'
        ORDER BY tmpM.Medarbejdernummer
    END

    -- Hvis man søger medarbejderens nummer
    ELSE
    BEGIN
        ;WITH
        AfdWithMng AS(
            SELECT 
                afd.AfdelingID,
                afd.Afdeling,
                ISNULL(mrb.Title + ' ', '') + ISNULL(mrb.Navn + ' ', '') + ISNULL(mrb.Middelnavn + ' ', '') + ISNULL(mrb.Efternavn + ' ', '') AS [LederensNavn]
            FROM [Eksamensopgave_jernbaneforvalter].[dbo].[afdelingtabel] AS afd
            LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[allemedarbejderetabel] AS mrb ON afd.Leder = mrb.Medarbejdernummer
        ),
        tempMedrbtabel AS(
            SELECT a.Medarbejdernummer, 
                ISNULL(a.Title + ' ', '') + ISNULL(a.Navn + ' ', '') + ISNULL(a.Middelnavn + ' ', '') + ISNULL(a.Efternavn + ' ', '') AS [Fuldnavn],
                j.Jobfunktion,
                afmgr.Afdeling,
                afmgr.LederensNavn
            FROM [Eksamensopgave_jernbaneforvalter].[dbo].[allemedarbejderetabel] AS a
            LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[jobfunktiontabel] AS j ON a.JobfunktionID = j.JobfunktionID
            LEFT OUTER JOIN AfdWithMng AS afmgr ON a.AfdelingID = afmgr.AfdelingID
        )
        SELECT tmpM.Medarbejdernummer, 
            tmpM.Fuldnavn,
            tmpM.Jobfunktion,
            tmpM.Afdeling,
            tmpM.LederensNavn
        FROM tempMedrbtabel AS tmpM
        WHERE tmpM.Medarbejdernummer = @Nummer
        ORDER BY tmpM.Medarbejdernummer

    END
GO

/* Hvis man vil søge medarbejderens navn */
EXEC stp_soegmedarbejder @Navn = 'Browning', @Nummer = 0;

/* Hvis man vil søge medarbejderens nummer */
--EXEC stp_soegmedarbejder @Navn = '', @Nummer = 5654;


Commands completed successfully.

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.011

Medarbejdernummer,Fuldnavn,Jobfunktion,Afdeling,LederensNavn
5302,Ms. Mary K. Browning,Spor-Ingeniør,Sporkonstruktioner,Mary E Gibson
5303,Mr. Dave P. Browning,El-Ingeniør,Signalsystemer,Sariya E Harnpadoungsataya


### 1.3.2 Se det totale antal arbejdstimer for hver måned for den pågældende medarbejder

In [96]:
-- Efter man har fundet medarbejderens nummer, kan man trække data for deres arbejdstimer hver måned

-- Drop the stored procedure called 'stp_medarbejderenstimer' in schema 'dbo'
USE [Eksamensopgave_jernbaneforvalter]

IF EXISTS (
SELECT *
    FROM INFORMATION_SCHEMA.ROUTINES
WHERE SPECIFIC_SCHEMA = N'dbo'
    AND SPECIFIC_NAME = N'stp_medarbejderenstimer'
)
DROP PROCEDURE dbo.stp_medarbejderenstimer
GO

-- Create a new stored procedure
CREATE PROCEDURE stp_medarbejderenstimer @Nummer INT, @aar INT
AS
    ;WITH 
    tmpMdrbTimer AS(
        SELECT Driftsrapportnummer, Antaltimer
        FROM [Eksamensopgave_jernbaneforvalter].[dbo].[medarbejdereperdriftstrapporttabel]
        WHERE Medarbejdernummer = @Nummer
    )
    SELECT 
        MONTH(adr.Udbedringsdato_og_tid) AS Maaned,
        --DATENAME(mm,adr.Udbedringsdato_og_tid) AS Maaned,
        SUM(tmpMT.Antaltimer) AS TotalAntalTimer
    FROM tmpMdrbTimer AS tmpMT
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[alledriftsrapportertabel] AS adr ON tmpMT.Driftsrapportnummer = adr.Driftsrapportnummer
    WHERE YEAR(Udbedringsdato_og_tid) = @aar
    GROUP BY MONTH(adr.Udbedringsdato_og_tid)
    ORDER BY MONTH(adr.Udbedringsdato_og_tid)
GO

/* Hvis man vil søge medarbejderens navn */
EXEC stp_soegmedarbejder @Navn = '', @Nummer = 5302;

EXEC stp_medarbejderenstimer @Nummer = 5302, @aar = 2017;



Commands completed successfully.

Commands completed successfully.

(1 row affected)

(10 rows affected)

Total execution time: 00:00:00.021

Medarbejdernummer,Fuldnavn,Jobfunktion,Afdeling,LederensNavn
5302,Ms. Mary K. Browning,Spor-Ingeniør,Sporkonstruktioner,Mary E Gibson


Maaned,TotalAntalTimer
1,4
2,2
3,1
4,5
5,9
6,2
7,30
8,1
9,18
10,7


## 1.4 Hvilke kompetencer har vi brugt mest i 2017?

In [100]:
-- Liste med alle kompetencer og det totale antal timer de var forbrugt

-- Drop the stored procedure called 'stp_kompetencertimer' in schema 'dbo'
USE [Eksamensopgave_jernbaneforvalter]

IF EXISTS (
SELECT *
    FROM INFORMATION_SCHEMA.ROUTINES
WHERE SPECIFIC_SCHEMA = N'dbo'
    AND SPECIFIC_NAME = N'stp_kompetencertimer'
)
DROP PROCEDURE dbo.stp_kompetencertimer
GO

-- Create a new stored procedure
CREATE PROCEDURE stp_kompetencertimer @aar INT
AS
    SELECT 
        jf.Jobfunktion,
        SUM(mdrbdrrp.Antaltimer) AS TotalAntalTimer
    FROM [Eksamensopgave_jernbaneforvalter].[dbo].[medarbejdereperdriftstrapporttabel] AS mdrbdrrp
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[alledriftsrapportertabel] AS adr ON mdrbdrrp.Driftsrapportnummer = adr.Driftsrapportnummer
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[allemedarbejderetabel] AS amdr ON mdrbdrrp.Medarbejdernummer = amdr.Medarbejdernummer
    LEFT OUTER JOIN [Eksamensopgave_jernbaneforvalter].[dbo].[jobfunktiontabel] AS jf ON amdr.JobfunktionID = jf.JobfunktionID
    WHERE YEAR(adr.Udbedringsdato_og_tid) = @aar
    GROUP BY jf.Jobfunktion
    ORDER BY TotalAntalTimer DESC
GO

EXEC stp_kompetencertimer @aar = 2017


(6 rows affected)

Total execution time: 00:00:00.021

Jobfunktion,TotalAntalTimer
El-Ingeniør,76347
IT-Ingeniør,75389
IT-tekniker,72289
Spor-tekniker,60110
El-tekniker,53095
Spor-Ingeniør,51724
